# GPR and Lidar

This notebook is designed to obtain snow depth, density, and SWE from ground-penetrating radar data. We will use GPR data gathered during the SnowEx Alaska campaigns, with validation from the airborne lidar obtained during the same campaign.

The text and code in this notebook is adapted from Randall Bonnell's tutorial on lidar and GPR during the 2024 SnowEx Hackweek, found here: https://snowex-2024.hackweek.io/tutorials/gpr_lidar/GPR_Lidar_HackweekTutorial.html

In [ ]:
!pip install -U snowexsql

![Title Card](./images/gpr_lidar/HackweekGraphicalAbstract_GPR_Lidar_v2.png)

# 1. GPR Methods for the Retrieval of Snow Depth and SWE

## What is GPR?
* GPR transmits a radar signal into the snowpack, which then reflects off objects/interfaces with contrasting dielectric permittivity. The GPR records the **amplitude** and **two-way travel time (twt)** of the reflections.
* **Dielectric permittivity** refers to the dielectric properties of the snowpack that define how EM energy trasmits through the medium.
* Usually, we are interested in the snow-ground interface, and we measure the snowpack thickness (depth) in two-way travel time (in nanoseconds).
* Most analysis-ready GPR products have twt, snow depth, and SWE variables. Some have also been updated to include derived snow density

For this notebook, we will start from the two-way travel time data to derive our snow properties of interest, and compare it to airborne lidar data. The `snowexsql` package will be needed.

Note that the results derived here are for the user's reference - most of the GPR products posted on NSIDC already have snow depth and SWE available as variables.

# 2. SnowEx23 GPR/Lidar Derived Permittivities/Densities in the Boreal Forest, Alaska

## Deriving Snow Density at Farmer's Loop/Creamer's Field
For this first step, we will use:
1. Airborne lidar data collected on 11 March, 2023.
2. GPR data collected on 7, 11, 13, and 16 March, 2023.

In [ ]:
#1.1 Load relevant packages
import os
import numpy as np 
from datetime import date
from scipy.spatial import cKDTree

#packages for figures
import matplotlib.pyplot as plt
from rasterio.plot import show

#geospatial packages
import geopandas as gpd #for vector data
import xarray as xr
import pandas as pd
from shapely.geometry import box, Point

import rasterio as rio

from snowexsql.lambda_client import SnowExLambdaClient


In [ ]:
# Initialize client
client = SnowExLambdaClient()

# Get all measurement classes dynamically
classes = client.get_measurement_classes()
LayerMeasurements = classes['LayerMeasurements']
PointMeasurements = classes['PointMeasurements']
RasterMeasurements = classes['RasterMeasurements']

print("🔍 Testing Lambda connection...")
connection_test = client.test_connection()
print(f"✅ Connected: {connection_test.get('connected', False)}")
if connection_test.get('connected'):
    print(f"📊 Database: {connection_test.get('version', 'Unknown version')}")
else:
    print("❌ Connection failed")


### Part 1: Load the GPR data from the SnowEx data base

In [ ]:
# Query all point measurements within 1000m of Creamer's Field center
# Creamer's Field coordinates: 64.864167°N, 147.737778°W (EPSG:4326)
# Converting to a Point geometry for the query

# Center point of Creamer's Field
creamers_field_center = Point(-147.737778, 64.864167)

# Query with 1000m buffer around the center point
df = PointMeasurements.from_area(
    pt=creamers_field_center,
    crs=4326,  # WGS84 coordinate system
    buffer=1000,  # 1000 meters
    type='two_way_travel',
    observer='Randall Bonnell',
    limit=20000,  # Adjust limit as needed
    verbose=False
)

In [ ]:
df.plot()

In [ ]:
# Let's look at the distribution of gpr two-way travel times and estimated snow depths
#Estimate snow depths from twt by assuming a velocity of 0.25 m/ns --> Is this an appropriate velocity estimate?
df['Depth_estimated'] = (df['value']/2)*0.25

ax1 = df.plot.hist(column=["value"], edgecolor='black', title='two-way travel time (ns)')
ax2 = df.plot.hist(column=["Depth_estimated"], edgecolor='black', title='Snow depth (m)')

### Load Lidar-Derived Canopy Height and Snow Depth
To compare against the GPR data, we will load airborne lidar data products, specifically canopy height and snow depths. To facilitate our analysis, we will create a bounding box for the lidar results using our GPR data.

In [ ]:
# Extract x/y limit from GPR data
bounds = df.total_bounds

# Create a bounding box
gpr_limits = box(*bounds)

Aside from this bounding box, accessing the lidar data is very similar to the GPR data. However, because the lidar data is much larger, loading it through the database may take a few minutes.

## Commented until we fix reading of raster data

In [ ]:
# Load lidar canopy heights
#flcf_ch = RasterMeasurements.from_area(shp=gpr_limits,
#                                      crs=26906,
#                                      buffer=None,
#                                      type='canopy_height',
#                                      site_name='farmers-creamers',
#                                      observers='chris larsen'
#                                      )
#print(flcf_ch)

In [ ]:
# Load lidar snow depths
#flcf_sd = RasterMeasurements.from_area(shp=gpr_limits,
#                                       crs=26906,
#                                       buffer=None,
#                                       type='depth',
#                                       site_name='farmers-creamers',
#                                       observers='chris larsen'
#                                      )
#print(flcf_sd)

### Match GPR data with lidar data
To match the GPR and lidar data, we can either rasterize the GPR data or vectorize the lidar data. For simplicity, we will vectorize the lidar data and perform a nearest neighbor search.

The GPR data is ~0.1 m resolution, whereas the lidar data is ~0.5 m resolution. So, we can expect ~5 GPR data points per lidar pixel.

In [ ]:
# # Find height and width of array
# height, width = flcf_sd.read(1).shape

# # Create arrays of the height, width of the lidar raster
# cols, rows = np.meshgrid(np.arange(width), np.arange(height))

# # Get the easting/northing from the lidar raster
# x_lidar, y_lidar = rio.transform.xy(flcf_sd.transform, rows, cols)

# # Vectorize the raster data
# x_lidar_vec = np.array(x_lidar).flatten()
# y_lidar_vec = np.array(y_lidar).flatten()
# flcf_sd_vec = flcf_sd.read().flatten()

# # Pull vectors from GPR GeoDataFrame
# gpr_arr = np.stack([gpr_df.geometry.x, gpr_df.geometry.y, gpr_df['value']], axis=1)
# gpr_x = gpr_arr[:,0]
# gpr_y = gpr_arr[:,1]
# gpr_twt = gpr_arr[:,2].reshape(len(gpr_arr[:,2]), 1)


For the nearest-neighbor approach, we will be using a K-D tree to efficiently find adjacent points.

In [ ]:
# # Create coordinate sets for nearest neighbor search
# set1 = np.column_stack((x_lidar_vec, y_lidar_vec))
# set2 = np.column_stack((gpr_x, gpr_y))

# # Build KDTree from GPR coordinates
# tree = cKDTree(set2)

# # Define the search radius (meters)
# radius = 0.25

# # Create function to find median travel times
# def median_travel_time(point, gpr_kdtree, gpr_coordinates, gpr_twt, radius):
#     indices = tree.query_ball_point(point, radius)
#     if indices:
#         # Retrieve travel times for the nearest neighbors
#         neighbor_twt = gpr_twt[indices]
#         median_twt = np.median(neighbor_twt)

#         return median_twt

#     else:
#         return np.nan

# # Find twt medians for each lidar point
# twt_median = np.array([median_travel_time(point,tree,set2,gpr_twt,radius) for point in set1])
# print(twt_median)


The GPR data is not as spatially continuous as the lidar data, so `twt_median` has a lot of NaN values. To reduce memory usage, let's remove those.

In [ ]:
# # Create mask for GPR NaN values
# mask = np.isnan(twt_median)

# # Mask GPR/lidar vectors
# flcf_sd_vec_clean = flcf_sd_vec[~mask]
# set1_clean = set1[~mask]
# twt_median_clean = twt_median[~mask]

# print(twt_median_clean.shape)

Now that the data is matched, and we have removed NaN values, we can now calculate relative permittivity and snow density.

In [ ]:
# # Calculate relative permittivity
# c = 0.2998 # Speed of light in a vaccuum
# e_s = ((c*twt_median_clean) / (2*flcf_sd_vec_clean))**2

# # Calculate snow density
# rho_s = ((np.sqrt(e_s)-1) / 0.845)*1000

### Examine the derived densities
Now that we've gone through all of this trouble, let's take a look at the derived snow densities.

In [ ]:
# # Plot snow density
# plt.figure()
# plt.scatter(set1_clean[:,0], set1_clean[:,1], s=10,
#             c=rho_s, cmap='viridis', clim=(0,500),
#             edgecolor=None
#            )
# plt.colorbar()
# plt.title('Snow density [kg m-3]')
# plt.show()

We can also check the histogram distribution of snow density.

In [ ]:
# # Define bin edges
# bin_edges = np.arange(np.min(rho_s), np.max(rho_s), 25)

# # Create the histogram
# plt.figure()
# plt.hist(rho_s, bins=bin_edges, edgecolor=None)
# plt.xlabel('Snow density [kg m-3]')
# plt.ylabel('Counts')
# plt.show()

There's some pretty unrealistic values in the data, so let's zoom in to more realistic densities.

In [ ]:
# # Re-define bin edges
# bin_edges = np.arange(0, 500, 25)

# # Create refined histogram
# plt.figure()
# plt.hist(rho_s, bins=bin_edges, edgecolor='black')
# plt.xlabel('Snow density [kg m-3]')
# plt.ylabel('Counts')
# plt.show()

Even at this range, there is a fair amount of random error. This may be caused by the measurement accuracy of the lidar or the GPR, the depth of the snow, and/or geolocation uncertainty. 

A user could further improve these densities by upsampling the lidar to match the GPR's geolocation uncertainty (3 m in this case), remove erroneous values in the snow depth or permittivity data, or run a spatial averaging filter.